In [1]:
import pandas as pd
import boto3
import botocore
from io import StringIO

class S3Utils:
    def __init__(self, file_path):
        self.file_path = file_path
        self.aws_access_key_id, self.aws_secret_access_key, self.bucket_name = self.get_aws_credentials()
        self.s3_client = self.create_s3_client()

    def get_aws_credentials(self):
        credentials_df = pd.read_csv(self.file_path)
        aws_access_key_id = credentials_df['aws_access_key_id'].iloc[0]
        aws_secret_access_key = credentials_df['aws_secret_access_key'].iloc[0]
        bucket_name = credentials_df['bucket_name'].iloc[0]
        return aws_access_key_id, aws_secret_access_key, bucket_name

    def create_s3_client(self):
        return boto3.client(
            's3',
            aws_access_key_id=self.aws_access_key_id,
            aws_secret_access_key=self.aws_secret_access_key
        )

    def check_and_transfer_file(self, source_key, destination_key):
        try:
            self.s3_client.head_object(Bucket=self.bucket_name, Key=destination_key)
        except botocore.exceptions.ClientError as e:
            if e.response['Error']['Code'] == "404":
                self.s3_client.copy_object(
                    Bucket=self.bucket_name,
                    CopySource=f"{self.bucket_name}/{source_key}",
                    Key=destination_key
                )
            else:
                raise e

    def check_file_exists(self, file_key):
        try:
            self.s3_client.head_object(Bucket=self.bucket_name, Key=file_key)
            return True
        except botocore.exceptions.ClientError as e:
            if e.response['Error']['Code'] == "404":
                return False
            else:
                raise e

    def read_csv_from_s3(self, file_key):
        obj = self.s3_client.get_object(Bucket=self.bucket_name, Key=file_key)
        return pd.read_csv(obj['Body'])

    def write_csv_to_s3(self, file_key, df):
        csv_buffer = StringIO()
        df.to_csv(csv_buffer, index=False)
        self.s3_client.put_object(
            Bucket=self.bucket_name,
            Key=file_key,
            Body=csv_buffer.getvalue()
        )
